# Research CoPilot
## Multimodal RAG with Code Execution (RAG-CE)

This notebook will demo the capabilities of the solution. In the below cells, we will ingest PDF documents that are located in the `cases` directory. Then we will generate questions based on **all** the information extracted from the 2 case documents. Then, with our multimodal Search that is supported by the Code Interpreter, we will generate potential answers to these questions and rate the ground truth vs the generated answer. 

This notebook is for demo purposes only, and therefore we will ignore the bias in the the below demo: by generating questions based on the extracted contents of the documents, this introduces a bias, since the questions are only limited to what the solution already has ingested. But again, this is for demo only. 

### Ingest the Directory
Import Python packages, and ingest the PDF directory.

In [3]:
%load_ext autoreload
%autoreload 2


import sys
sys.path.append('..\\code')


import os
from dotenv import load_dotenv
load_dotenv()

from IPython.display import display, Markdown, HTML
from PIL import Image
from doc_utils import *
from processor import *


def show_img(img_path, width = None):
    if width is not None:
        display(HTML(f'<img src="{img_path}" width={width}>'))  
    else:
        display(Image.open(img_path))




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
doc_path = r"sample_data\minion-tech.pdf"

ingestion_params_dict = {
        "index_name" : 'project_minions',
        "delete_existing_output_dir": True,
        "processing_mode_pdf" : 'hybrid',
        "doc_path" : doc_path,
        'num_threads': 1
    }

pdf1 = PdfProcessor(ingestion_params_dict)
pdf1.ingest_document()


doc_path = r"sample_data\wile_e_coyote.pdf"

ingestion_params_dict = {
        "index_name" : 'project_minions',
        "delete_existing_output_dir": False,
        "processing_mode_pdf" : 'hybrid',
        "doc_path" : doc_path,
        'num_threads': 1
    }

pdf2 = PdfProcessor(ingestion_params_dict)
pdf2.ingest_document()

### Genereate questions based on the ingested output of the case documents

The below uses only 5 prompts to generate questions / answers pairs that are considered "ground truth" answers.

In [2]:
path_1 = r"..\project_minions\minion-tech.pdf\minion-tech.txt"
path_2 = r"..\project_minions\wile_e_coyote.pdf\wile_e_coyote.txt"

text_template = """
Gru's Document: Business Analysis Document for Minion Tech
## START OF GRU'S DOCUMENT
{gru_doc}
## END OF GRU'S DOCUMENT



Wile E. Coyote's Document: Investment Proposal for Gru's Enterprises
## START OF WILE E. COYOTE'S DOCUMENT
{wile_doc}
## END OF WILE E. COYOTE'S DOCUMENT

"""

text = text_template.format(gru_doc = read_asset_file(path_1)[0] , wile_doc = read_asset_file(path_2)[0])

past_questions = []

qna_pairs = recover_json(ask_LLM_with_JSON(general_prompt_template.format(context=text, past_questions=past_questions), temperature = 0.5))
past_questions.extend(qna_pairs['qna_pairs'])
logc("General Q&A", json.dumps(qna_pairs, indent=4))

qna_pairs = recover_json(ask_LLM_with_JSON(specialized_prompt_template.format(context=text, past_questions=past_questions), temperature = 0.5))
past_questions.extend(qna_pairs['qna_pairs'])
logc("Specialized Q&A", json.dumps(qna_pairs, indent=4))

qna_pairs = recover_json(ask_LLM_with_JSON(numerical_prompt_template.format(context=text, past_questions=past_questions), temperature = 0.5))
past_questions.extend(qna_pairs['qna_pairs'])
logc("Numerical Q&A", json.dumps(qna_pairs, indent=4))

qna_pairs = recover_json(ask_LLM_with_JSON(table_prompt_template.format(context=text, past_questions=past_questions), temperature = 0.5))
past_questions.extend(qna_pairs['qna_pairs'])
logc("Tables Q&A", json.dumps(qna_pairs, indent=4))
    
qna_pairs = recover_json(ask_LLM_with_JSON(image_prompt_template.format(context=text, past_questions=past_questions), temperature = 0.5))
past_questions.extend(qna_pairs['qna_pairs'])
logc("Images Q&A", json.dumps(qna_pairs, indent=4))
    


11.02.2024_21.17.35 :: General Q&A: {
    "qna_pairs": [
        {
            "question": "What is the projected increase in annual revenue for Gru's Enterprises over the next three years according to the investment appeal document?",
            "answer": "Gru's Enterprises projects a 25% increase in annual revenue over the next three years."
        },
        {
            "question": "What is the total valuation of Gru's Enterprises as calculated in Wile E. Coyote's investment proposal?",
            "answer": "The total valuation of Gru's Enterprises is calculated to be $5 million."
        },
        {
            "question": "What are the main components of the manufacturing process at Gru's Enterprises as detailed in their business analysis document?",
            "answer": "The main components of the manufacturing process at Gru's Enterprises are Design and Prototyping, Material Sourcing, Assembly, and Testing."
        }
    ]
}

11.02.2024_21.17.57 :: Specialized Q&A: {
  

### Generate answers

With our multimodal Search that is potentially supported by the Code Interpreter, we will generate answers to the questions and rate the ground truth vs the generated answer. 

In [3]:
for index, qna_pair in enumerate(past_questions):
    gen_answer, references, _, _ = search(
        qna_pair['question'], 
        top=3, 
        computation_approach = "Taskweaver",  ## other options are "NoComputationTextOnly", "Taskweaver", "AssistantsAPI", or "LocalPythonExec"
        computation_decision = "LLM", 
        index_name = index_name,
        vision_support = False,  
        count = False, 
        verbose = False)

    rating = recover_json(ask_LLM_with_JSON(rate_answers_prompt_template.format(question=qna_pair['question'], ground_truth_answer=qna_pair['answer'], generated_answer=gen_answer)))
    rating = int(rating['rating'])
    past_questions[index]['rating'] = rating
    logc("Rating for question " + str(index) + " is --> " + str(rating), 
         f"\nQuestion: {qna_pair['question']}\n{bc.OKCYAN}Ground truth answer: {qna_pair['answer']}\n{bc.MAGENTA}Generated answer: {gen_answer}\n{bc.LIGHT_RED}References: {json.dumps(references, indent=4)}\n\n\n")




11.02.2024_21.22.54 :: Rating for question 0 is --> 0: 
Question: What is the projected increase in annual revenue for Gru's Enterprises over the next three years according to the investment appeal document?
Ground truth answer: Gru's Enterprises projects a 25% increase in annual revenue over the next three years.
Generated answer: The projected increase in annual revenue for Gru's Enterprises over the next three years is calculated based on the provided revenue figures for each year. The computation is as follows:

- The projected revenue for Year 1 (2024) is $5,500,000.
- The projected revenue for Year 2 (2025) is $6,500,000.
- The projected revenue for Year 3 (2026) is $7,500,000.

To calculate the increase from Year 1 to Year 2, we subtract the Year 1 revenue from the Year 2 revenue:
$6,500,000 (Year 2) - $5,500,000 (Year 1) = $1,000,000

Similarly, to calculate the increase from Year 2 to Year 3, we subtract the Year 2 revenue from the Year 3 revenue:
$7,500,000 (Year 3) - $6,500